In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("cannabisroad2")

create_database('blp_dataset')
create_table('products_tmp_cannabisroad')

In [ ]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("pre")

                if raw_desc is not None:
                    description = get_content(raw_desc)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("title")
                
                if raw_title is not None:
                    title = get_content(raw_title)
                    
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("a", attrs={"class": "toplink"})
                
                if raw_seller is not None:
                    seller = get_content(raw_seller)
                    
                ## GET PRICE
                price = ""
                raw_price = page_content.find("span", attrs={"class": "price"})
                
                if raw_price is not None:
                    price = get_content(raw_price)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_cannabisroad (name, description, market_name, price, seller_name, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                    values = (title, description, "Cannabis Road 2", price, seller, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    raw_image = page_content.find("div", attrs={"class": "product-img-box"})
                    
                    if raw_image is not None:
                        img_content = raw_image.find("img")

                        if img_content is not None:
                            image = img_content["src"]

                            try:
                                data = "" + image.replace("data:image/jpeg;base64,", "")
                                imgdata = base64.b64decode(data)

                                img_filename = Path(f"D:\images\_cannabis_road_2\{product_id}.jpeg")
                                
                                print(f"{product_id}")
                                print(f"{folder_image}\{file}")

                                with open(img_filename, 'wb') as f:
                                    f.write(imgdata)
                                    f.close
                            except:
                                print(f"Image {product_id} has error. Except")
                                print(f"{folder_image}\{file}")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Product {product_id} has no image")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image") 